In [1]:
import torch
import torch.nn as nn

In [21]:
class AutoRec(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AutoRec, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)
        self.act = nn.Sigmoid()
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.act(x)
        x = self.decoder(x)
        return x

In [ ]:
def loss_AutorRec(outputs, targets, parameters, l2_weight):
    mse_fn = nn.MSELoss()
    mse = mse_fn(torch.where(targets != 0, outputs, 0.), targets)
    l2 = 0
    for param in parameters:
        l2 += torch.norm(param)**2
    return mse + l2_weight*l2

In [2]:
# Gradient Reversal Layer
class GRL(torch.autograd.Function):
    @staticmethod
    def forward(context, x, constant):
        context.constant = constant
        return x.view_as(x)*constant

    @staticmethod
    def backward(context, grad):
        return grad.neg()*context.constant, None

In [10]:
class DARec(nn.Module):
    def __init__(self,
                 input_dim, hidden_dim,
                 RP_dim1, RP_dim2, output_dim,
                 GRL_scale,
                 DC_dim):
        super(DARec, self).__init__()
        # Rating Pattern Extractor
        self.RPE = nn.Linear(input_dim, hidden_dim)
        # Rating Predictor for Domain1
        self.RP1_ln1 = nn.Linear(hidden_dim, RP_dim1)
        self.RP1_ln2 = nn.Linear(RP_dim1, RP_dim2)
        self.RP1_ln3 = nn.Linear(RP_dim2, output_dim)
        # Rating Predictor for Domain2
        self.RP2_ln1 = nn.Linear(hidden_dim, RP_dim1)
        self.RP2_ln2 = nn.Linear(RP_dim1, RP_dim2)
        self.RP2_ln3 = nn.Linear(RP_dim2, output_dim)
        # Gradient Reverse Layer
        self.GRL_scale = torch.tensor(GRL_scale)
        # Domain Classifier
        self.DC_ln1 = nn.Linear(hidden_dim, DC_dim)
        self.DC_ln2 = nn.Linear(DC_dim, 2)
        # activation function
        self.act = nn.ReLU()
        
    def forward(self, x, flag):
        x = self.RPE(x)
        x = self.act(x)
        
        x_hat1 = self.RP1_ln1(x)
        x_hat1 = self.act(x_hat1)
        x_hat1 = self.RP1_ln2(x_hat1)
        x_hat1 = self.act(x_hat1)
        x_hat1 = self.RP1_ln3(x_hat1)
        
        x_hat2 = self.RP2_ln1(x)
        x_hat2 = self.act(x_hat2)
        x_hat2 = self.RP2_ln2(x_hat2)
        x_hat2 = self.act(x_hat2)
        x_hat2 = self.RP2_ln3(x_hat2)
        
        x_class = GRL.apply(x, self.GRL_scale)
        x_class = self.DC_ln1(x_class)
        x_class = self.act(x_class)
        x_class = self.DC_ln2(x_class)
        
        return x_hat1, x_hat2, x_class

In [ ]:
def loss_DARec(rating_pred1, rating_targets1,
               rating_pred2, rating_targets2, RPloss_weight,
               domain_pred, domain_targets, DCloss_weight,
               parameters, l2_weight):
    def loss_predictor(outputs, targets):
        mse_fn = nn.MSELoss()
        mse = mse_fn(torch.where(targets != 0, outputs, 0.), targets)
        return mse
    loss_pred1 = loss_predictor(rating_pred1, rating_targets1)
    loss_pred2 = loss_predictor(rating_pred2, rating_targets2)
    
    crossentropy_fn = nn.CrossEntropyLoss()
    loss_classisfy = crossentropy_fn(domain_pred, domain_targets)
    
    l2 = 0
    for param in parameters:
        l2 += torch.norm(param)**2
    
    return loss_pred1 + RPloss_weight*loss_pred2 - DCloss_weight*loss_classisfy + l2_weight*l2